# 数据的读取与存储

In [3]:
# Creating a SparkSession in Python
from pyspark.sql import SparkSession
# getOrCreate()如果存在则获取，如果不存在则创建
spark = SparkSession.builder.master("local").appName("Spark Reading and Loading").getOrCreate() 

## 文本文件
使用SparkContext中的textFile()函数，就可以读取一个文本文件。

In [4]:
sale_rdd = spark.sparkContext.textFile('./data/sales/sales_1')
sale_rdd.collect()

['iphone,8000', 'iwhach,4000', 'pro,10000', 'pro,10000']

如果多个输入文件存放在一个目录下，仍然可以用textFile将所有数据都读入的rdd中

In [7]:
sale_rdd = spark.sparkContext.textFile('./data/sales')
sale_rdd.collect()

['iphone,8000',
 'iwhach,4000',
 'ipad,3000',
 'ipad,3000',
 'iphone,8000',
 'iwhach,4000',
 'pro,10000',
 'pro,10000']

对于多个文件存放在一个路目录下，我们又想知道哪些数据来自于哪个文件，<br>
我们可以使用wholeTextFiles()把所有目录下所有数据读入到一个键值对RDD中。<br>
key为文件名<br>
但最好用于小文件

In [6]:
sale_rdd = spark.sparkContext.wholeTextFiles('./data/sales')
sale_rdd.collect()

[('file:/home/jovyan/work/scripts/data/sales/sales_2',
  'iphone,8000\niwhach,4000\nipad,3000\nipad,3000\n'),
 ('file:/home/jovyan/work/scripts/data/sales/sales_1',
  'iphone,8000\niwhach,4000\npro,10000\npro,10000\n')]

使用RDD的saveAsTextFile()方法，可以将RDD保存到对应的路径中<br>
saveAsTextFile()接受一个参数，spark将该参数当作**路径** ,会在该路径下生成多个文件，<br>
这样，spark就会在多个路径下并行的输出了，spark并不能控制哪一部分输出到哪个文件

In [9]:
sale_rdd.saveAsTextFile('./data/sale_output')

读取json的一种方式是，按照读普通文本的方式将数据都加载到rdd中，使用python、scala或java中的json解释器进行解析

In [30]:
import json
sale_rdd = spark.sparkContext.textFile('./data/people.json')
sale_rdd.map(lambda x: json.loads(x)).collect()


[{'name': 'Michael'},
 {'name': 'Andy', 'age': 30},
 {'name': 'Justin', 'age': 19}]

In [ ]:
或者先加载到DataFrame中再转换为RDD

In [15]:
df_json = spark.read.json("./data/people.json")

In [20]:
df_json.rdd.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [21]:
df_json.rdd.filter(lambda x: x['name'] == 'Andy').collect()

[Row(age=30, name='Andy')]

In [22]:
df_json.rdd.filter(lambda x: x[1] == 'Andy').collect()

[Row(age=30, name='Andy')]

将RDD导出到json文件，可以使用python、scala或者java中内置的json处理方法，将数据转换为json格式

In [32]:
sale_rdd = spark.sparkContext.textFile('./data/sales/sales_1')
sale_rdd.collect()

['iphone,8000', 'iwhach,4000', 'pro,10000', 'pro,10000']

In [33]:
import json
def convert_to_dict(x):
    res = {}
    res['name'] = x.split(',')[0]
    res['price'] = x.split(',')[1]
    return res
    
sale_rdd_json = sale_rdd.map(convert_to_dict)
sale_rdd_json.collect()

[{'name': 'iphone', 'price': '8000'},
 {'name': 'iwhach', 'price': '4000'},
 {'name': 'pro', 'price': '10000'},
 {'name': 'pro', 'price': '10000'}]

In [34]:
sale_rdd_json.map(lambda x: json.dumps(x)).saveAsTextFile('./data/sale_json')

读取csv文件

In [5]:
sale_rdd = spark.sparkContext.textFile('./data/people.csv')
sale_rdd.collect()

['name,age,job', 'Jorge,30,Developer', 'Bob,32,Developer']

In [6]:
sale_rdd.map(lambda x: x.split(',')).collect()

[['name', 'age', 'job'],
 ['Jorge', '30', 'Developer'],
 ['Bob', '32', 'Developer']]

读取到DataFrame中

In [11]:
sale_rdd = spark.read.csv('./data/people.csv', sep = ',', header = True)

In [12]:
sale_rdd.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [13]:
sale_rdd.rdd.collect()

[Row(name='Jorge', age='30', job='Developer'),
 Row(name='Bob', age='32', job='Developer')]